Submitting various things for end of grant.

In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
PANDAS_ODF = os.path.expanduser('~/src/pandasodf')
if PANDAS_ODF not in sys.path:
    sys.path.append(PANDAS_ODF)
    from pandasodf import ODFReader

In [4]:
from htsworkflow.submission.encoded import Document, typed_column_parser
from htsworkflow.submission.aws_submission import run_aws_cp

In [5]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-tranche1-resubmit.ods')

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-tranche1-resubmit-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [6]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

In [7]:
#atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
#atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
#                'general protocol',
#                'ATAC-Seq experiment protocol for Wold lab',
#                )
#body = atac.create_if_needed(server, atac_uuid)
#print(body['@id'])

# Submit Annotations

In [8]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [9]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [26]:
book = ODFReader(spreadsheet_name)
biosample = book.parse('Biosample', header=0)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [25]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [29]:
print(spreadsheet_name)
book = ODFReader(spreadsheet_name)
libraries = book.parse('Library', header=0)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

/home/diane/woldlab/ENCODE/C1-encode3-limb-tranche1-resubmit.ods
0


In [28]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

In [14]:
len(created)

8

# Register Experiments

In [32]:
book = ODFReader(spreadsheet_name)
experiments = book.parse('Experiment', header=0)
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [31]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [35]:
book = ODFReader(spreadsheet_name)
replicates = book.parse('Replicate', header=0)
created = server.post_sheet('/replicates/', 
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [34]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# Find size_range

In [19]:
from rdflib import Graph
from rdflib.term import URIRef
from rdflib.serializer import Serializer
import json
import numpy

In [20]:
print(spreadsheet_name)
book = ODFReader(spreadsheet_name)
libraries = book.parse('Library', header=0)

/home/diane/woldlab/ENCODE/C1-encode3-limb-tranche1-resubmit.ods


In [21]:
libraries

,uuid,library_accession,aliases:array,biosample,nucleic_acid_term_name,depleted_in_term_name,nucleic_acid_starting_quantity,nucleic_acid_starting_quantity_units,size_range,lysis_method,...,extraction_method,library_size_selection_method,treatments,strand_specificity:boolean,spikeins_used:array,spike in pool:skip,documents:array,lab,award,barcode_details:json
0,NaN,NaN,"barbara-wold:18042_A1,barbara-wold:18042_A11,b...",barbara-wold:c1_e10.5_mouse_limb_run2_template,polyadenylated mRNA,NaN,NaN,NaN,493-662,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_2,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""CGAGGCTGGCGTAAGA"", ""plate_id"": ""..."
1,NaN,NaN,"barbara-wold:17327_A10,barbara-wold:17327_A11,...",barbara-wold:c1_e10.5_mouse_limb_run1_template,polyadenylated mRNA,NaN,NaN,NaN,309-365,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_1,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""TAAGGCGAGCGTAAGA"", ""plate_id"": ""..."
2,NaN,NaN,"barbara-wold:18087_F1,barbara-wold:18087_F10,b...",barbara-wold:c1_e10.5_mouse_limb_run3_template,polyadenylated mRNA,NaN,NaN,NaN,472-554,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""TAAGGCGAACTGCATA"", ""plate_id"": ""..."
3,NaN,NaN,"barbara-wold:18258_A1,barbara-wold:18258_A10,b...",barbara-wold:c1_e11.0_mouse_limb_run5_template,polyadenylated mRNA,NaN,NaN,NaN,287-347,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""TAAGGCGAGCGTAAGA"", ""plate_id"": ""..."
4,NaN,NaN,"barbara-wold:18263_A10,barbara-wold:18263_A11,...",barbara-wold:c1_e11.5_mouse_limb_run6_template,polyadenylated mRNA,NaN,NaN,NaN,244-301,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""CGATCAGTTCGACTAG"", ""plate_id"": ""..."
5,NaN,NaN,"barbara-wold:18270_A1,barbara-wold:18270_A10,b...",barbara-wold:c1_e12.5_mouse_limb_run7_template,polyadenylated mRNA,NaN,NaN,NaN,425-519,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""ACTCGCTATCGACTAG"", ""plate_id"": ""..."
6,NaN,NaN,"barbara-wold:18251_A1,barbara-wold:18251_A10,b...",barbara-wold:c1_e13.5_mouse_limb_run4_template,polyadenylated mRNA,NaN,NaN,NaN,452-516,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""TAAGGCGAGCGTAAGA"", ""plate_id"": ""..."
7,NaN,NaN,"barbara-wold:18311_A1,barbara-wold:18311_A12,b...",barbara-wold:c1_e13.5_mouse_limb_run8_template,polyadenylated mRNA,NaN,NaN,NaN,387-466,NaN,...,NaN,NaN,NaN,NaN,barbara-wold:profile_C1_3,NaN,barbara-wold:smarter-ulta-low-pdf,barbara-wold,UM1HG009443,"[{""barcode"": ""ACTCGCTACTCTCTAT"", ""plate_id"": ""..."


In [22]:
def get_size_range(library_ids):
    sizes = []
    for library_id in library_ids:
        g = Graph()
        g.parse('https://jumpgate.caltech.edu/library/{}/'.format(library_id), format='rdfa')
        for row in g.subject_objects(URIRef('http://jumpgate.caltech.edu/wiki/LibraryOntology#insert_size')):
            subject, value = row
            #print(subject, value)
            sizes.append(value.toPython())
    return (min(sizes), max(sizes))
    

In [23]:
for i, row in libraries[['library_accession', 'barcode_details:json']].iterrows():
    library_ids = set()
    for cell in json.loads(row['barcode_details:json']):
        library_ids.add(cell['plate_id'])
    #print(library_ids)
    low, high = get_size_range(library_ids)
    print(row.library_accession, '{}-{}'.format(low, high))

nan 493-662
nan 309-365
nan 472-554
nan 287-347
nan 244-301
nan 425-519
nan 452-516
nan 387-466
